# Accessing OpenBIDS for CML Data
This tutorial shows how to load behavioral and EEG data in OpenBIDS format so it matches the structure used by the Computational Memory Lab’s CMLReader package. The structure is designed to support new analyses and replicate existing scripts with minimal changes.

In [1]:
# imports
import pandas as pd; pd.set_option('display.max_columns', None)
import cmlreaders as cml
import numpy as np
import xarray as xr
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals
import os
import mne
from ptsa.data.timeseries import TimeSeries

## Loading Behavioral Data

### CMLReader

As a reminder, to load behavioral data for a given experimental session, we select the session using its subject, experiment, and session identifiers, instantiate a CMLReader object, and then load the desired data type (e.g., events). 

In [2]:
# load dataframe of all sessions
df = cml.get_data_index()
df[:10]                     # show the first 10 entries

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/ltp/subjects/LTP001/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,0,NaN,NaN,0,LTP001,LTP001,NaN,protocols/ltp/subjects/LTP001/experiments/Valu...
1,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,0,NaN,NaN,0,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
2,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,1,NaN,NaN,1,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
3,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,10,NaN,NaN,10,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
4,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,11,NaN,NaN,11,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
5,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,12,NaN,NaN,12,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
6,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,13,NaN,NaN,13,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
7,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,14,NaN,NaN,14,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
8,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,15,NaN,NaN,15,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...
9,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...,NaN,ltpFR,build,0,protocols/ltp/subjects/LTP063/experiments/ltpF...,0,NaN,16,NaN,NaN,16,LTP063,LTP063,NaN,protocols/ltp/subjects/LTP063/experiments/ltpF...


In [3]:
df_PS2 = df.query("experiment == 'FR1'").iloc[0]
df_PS2

Recognition                                                          NaN
all_events             protocols/r1/subjects/R1001P/experiments/FR1/s...
contacts               protocols/r1/subjects/R1001P/localizations/0/m...
experiment                                                           FR1
import_type                                                        build
localization                                                           0
math_events            protocols/r1/subjects/R1001P/experiments/FR1/s...
montage                                                                0
original_experiment                                                  NaN
original_session                                                       0
pairs                  protocols/r1/subjects/R1001P/localizations/0/m...
ps4_events                                                           NaN
session                                                                0
subject                                            

In [4]:
cmlreader = cml.CMLReader(subject='R1111M', experiment='FR1', session=0)
# load the behavioral events
evs_PS2 = cmlreader.load('pairs')

In [5]:
evs_PS2

,contact_1,contact_2,label,id,is_explicit,is_stim_only,type_1,type_2,avg.region,avg.x,avg.y,avg.z,avg.dural.region,avg.dural.x,avg.dural.y,avg.dural.z,dk.region,dk.x,dk.y,dk.z,ind.region,ind.x,ind.y,ind.z,ind.dural.region,ind.dural.x,ind.dural.y,ind.dural.z,tal.region,tal.x,tal.y,tal.z,wb.region,wb.x,wb.y,wb.z,stein.region,stein.x,stein.y,stein.z,mni.x,mni.y,mni.z
0,1,9,LPOG1-LPOG9,lpog.1-lpog.9,False,False,G,G,middletemporal,-65.295,-24.035,-25.980,inferiortemporal,-61.023932,-23.512013,-24.379970,middletemporal,NaN,NaN,NaN,middletemporal,-58.010,-8.475,-41.135,middletemporal,-57.664615,-8.899387,-41.833920,None,-66.60470,-25.48000,-20.609300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-65.28000,-24.479300,-25.704682
1,1,2,LPOG1-LPOG2,lpog.1-lpog.2,False,False,G,G,middletemporal,-66.140,-18.330,-21.425,middletemporal,-64.503449,-17.996502,-19.586545,middletemporal,NaN,NaN,NaN,middletemporal,-58.625,-2.740,-37.385,middletemporal,-57.881678,-1.366716,-36.869017,None,-67.64310,-19.84015,-17.089950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-67.32435,-20.277700,-20.704405
2,2,10,LPOG2-LPOG10,lpog.2-lpog.10,False,False,G,G,middletemporal,-67.055,-23.345,-15.730,middletemporal,-66.054467,-23.321447,-15.071706,middletemporal,NaN,NaN,NaN,middletemporal,-59.745,-7.220,-32.025,middletemporal,-58.843694,-6.011109,-31.030316,None,-68.34015,-24.39265,-11.990150,NaN,NaN,NaN,NaN,Left Middle Temporal Gyrus,NaN,NaN,NaN,-68.17790,-24.273700,-16.147723
3,2,3,LPOG2-LPOG3,lpog.2-lpog.3,False,False,G,G,middletemporal,-66.335,-17.670,-10.835,middletemporal,-65.226696,-17.899205,-9.973574,middletemporal,NaN,NaN,NaN,middletemporal,-58.890,-1.560,-27.950,middletemporal,-58.147129,-0.025947,-27.633047,None,-67.76490,-18.65010,-8.186165,NaN,NaN,NaN,NaN,Left Middle Temporal Gyrus,NaN,NaN,NaN,-67.85235,-18.974050,-11.785856
4,3,4,LPOG3-LPOG4,lpog.3-lpog.4,False,False,G,G,superiortemporal,-63.665,-17.575,0.420,superiortemporal,-63.070961,-17.929364,0.861704,superiortemporal,NaN,NaN,NaN,superiortemporal,-56.485,-0.990,-17.850,superiortemporal,-57.331785,-0.860819,-16.950674,None,-64.91250,-17.74555,1.544220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-65.81390,-16.246800,-0.170758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,94,95,LPS2-LPS3,lps.2-lps.3,False,False,S,S,inferiortemporal,-48.050,-2.420,-46.040,inferiortemporal,-44.934177,-5.384076,-43.312384,inferiortemporal,NaN,NaN,NaN,inferiortemporal,-40.720,10.540,-60.025,inferiortemporal,-40.865877,10.696948,-59.814793,None,-49.88840,-4.68133,-38.805100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-50.32230,-1.714042,-45.288351
137,95,96,LPS3-LPS4,lps.3-lps.4,False,False,S,S,middletemporal,-55.070,-1.090,-38.320,middletemporal,-51.923399,-3.531084,-36.772209,middletemporal,NaN,NaN,NaN,middletemporal,-47.350,12.485,-53.290,middletemporal,-47.618277,12.176917,-52.769062,None,-57.07530,-3.48274,-32.357500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-57.81460,-0.218944,-36.750696
138,97,98,LTD1-LTD2,ltd.1-ltd.2,False,False,D,D,parahippocampal,-25.825,-23.720,-24.560,parahippocampal,-25.825000,-23.720000,-24.560000,parahippocampal,NaN,NaN,NaN,parahippocampal,-20.875,-9.625,-39.345,parahippocampal,-20.875000,-9.625000,-39.345000,None,-26.04175,-22.31570,-19.469800,Left Cerebral White Matter,NaN,NaN,NaN,Left MTL WM,NaN,NaN,NaN,-26.55900,-19.346450,-24.104513
139,98,99,LTD2-LTD3,ltd.2-ltd.3,False,False,D,D,None,-30.450,-21.155,-23.910,None,-30.450000,-21.155000,-23.910000,NaN,NaN,NaN,NaN,parahippocampal,-25.130,-6.950,-38.970,parahippocampal,-25.130000,-6.950000,-38.970000,None,-30.88385,-20.15175,-19.065000,Left Cerebral White Matter,NaN,NaN,NaN,Left PRC,NaN,NaN,NaN,-30.96920,-18.622350,-25.020927


In [7]:
root = "/data/LTP_BIDS/FR1"

base = BIDSPath(
    subject="R1111M",
    session="0",
    task="FR1",
    datatype="ieeg",
    root=root,
    check=False
)


# print(bp.fpath)
# electrodes
elec = pd.read_csv(
    base.copy().update(
        suffix="electrodes",
        extension=".tsv",
        space="MNI152NLin6ASym"
    ).fpath,
    sep="\t"
)

elec

,name,x,y,z,size,group,hemisphere,type,tal.x,tal.y,tal.z,wb.region,ind.region,stein.region
0,LPOG1,-67.9554,-20.436300,-26.318920,-999,LPOG,L,grid,-66.7592,-20.37470,-21.06940,NaN,middletemporal,NaN
1,LPOG2,-71.3723,-19.887300,-17.033223,-999,LPOG,L,grid,-68.5270,-19.30560,-13.11050,NaN,middletemporal,NaN
2,LPOG3,-69.4694,-16.873900,-5.837007,-999,LPOG,L,grid,-67.0028,-17.99460,-3.26183,NaN,middletemporal,NaN
3,LPOG4,-68.4177,-13.619100,6.599195,-999,LPOG,L,grid,-62.8222,-17.49650,6.35027,NaN,superiortemporal,NaN
4,LPOG5,-68.5695,-14.288000,16.220750,-999,LPOG,L,grid,-60.2654,-16.09750,16.38480,NaN,postcentral,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LPS4,-60.3150,0.724204,-33.789406,-999,LPS,L,strip,-59.5567,-3.04176,-28.33260,NaN,middletemporal,NaN
96,LTD1,-24.4314,-20.139100,-23.616084,-999,LTD,L,depth,-23.8723,-23.65220,-19.49340,Left PHG parahippocampal gyrus,parahippocampal,Left EC
97,LTD2,-28.6866,-18.553800,-24.592941,-999,LTD,L,depth,-28.2112,-20.97920,-19.44620,Left Cerebral White Matter,parahippocampal,Left MTL WM
98,LTD3,-33.2518,-18.690900,-25.448912,-999,LTD,L,depth,-33.5565,-19.32430,-18.68380,Left PHG parahippocampal gyrus,parahippocampal,Left PRC


In [8]:
# load channel definitions

ch_mono = pd.read_csv(
    base.copy().update(
        acquisition="monopolar",
        suffix="channels",
        extension=".tsv"
    ).fpath,
    sep="\t"
)

ch_bip = pd.read_csv(
    base.copy().update(
        acquisition="bipolar",
        suffix="channels",
        extension=".tsv"
    ).fpath,
    sep="\t"
)


In [9]:
ch_bip

,name,type,units,low_cutoff,high_cutoff,reference,group,sampling_frequency,description,notch
0,LPOG1-LPOG9,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
1,LPOG1-LPOG2,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
2,LPOG2-LPOG10,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
3,LPOG2-LPOG3,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
4,LPOG3-LPOG4,ECOG,V,NaN,NaN,bipolar,LPOG,500,grid,NaN
...,...,...,...,...,...,...,...,...,...,...
136,LPS2-LPS3,ECOG,V,NaN,NaN,bipolar,LPS,500,strip,NaN
137,LPS3-LPS4,ECOG,V,NaN,NaN,bipolar,LPS,500,strip,NaN
138,LTD1-LTD2,SEEG,V,NaN,NaN,bipolar,LTD,500,depth,NaN
139,LTD2-LTD3,SEEG,V,NaN,NaN,bipolar,LTD,500,depth,NaN


In [10]:
ch_mono

,name,type,units,low_cutoff,high_cutoff,group,sampling_frequency,description,notch
0,LPOG1,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
1,LPOG2,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
2,LPOG3,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
3,LPOG4,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
4,LPOG5,ECOG,V,NaN,NaN,LPOG,500,grid,NaN
...,...,...,...,...,...,...,...,...,...
95,LPS4,ECOG,V,NaN,NaN,LPS,500,strip,NaN
96,LTD1,SEEG,V,NaN,NaN,LTD,500,depth,NaN
97,LTD2,SEEG,V,NaN,NaN,LTD,500,depth,NaN
98,LTD3,SEEG,V,NaN,NaN,LTD,500,depth,NaN


In [11]:
# load mono eeg
raw_mono = read_raw_bids(
    base.copy().update(
        acquisition="monopolar",
        suffix="ieeg",
        extension=".edf"
    )
)
raw_mono

Extracting EDF parameters from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-monopolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_events.tsv.
Reading channel info from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-monopolar_channels.tsv.
Reading electrode coords from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_space-MNI152NLin6ASym_electrodes.tsv.


/tmp/ipykernel_103803/771245736.py:2: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw_mono = read_raw_bids(
/tmp/ipykernel_103803/771245736.py:2: RuntimeWarning: MNI152NLin6ASym is not an MNE-Python coordinate frame for IEEG data and so will be set to 'unknown'
  raw_mono = read_raw_bids(


<RawEDF | sub-R1111M_ses-0_task-FR1_acq-monopolar_ieeg.edf, 100 x 1623500 (3247.0 s), ~117 kB, data not loaded>

In [12]:
# load bipolar eeg
raw_bip = read_raw_bids(
    base.copy().update(
        acquisition="bipolar",
        suffix="ieeg",
        extension=".edf"
    )
)
raw_bip

Extracting EDF parameters from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_events.tsv.
Reading channel info from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_acq-bipolar_channels.tsv.
Reading electrode coords from /data/LTP_BIDS/FR1/sub-R1111M/ses-0/ieeg/sub-R1111M_ses-0_task-FR1_space-MNI152NLin6ASym_electrodes.tsv.


/tmp/ipykernel_103803/1923545361.py:2: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw_bip = read_raw_bids(
/tmp/ipykernel_103803/1923545361.py:2: RuntimeWarning: MNI152NLin6ASym is not an MNE-Python coordinate frame for IEEG data and so will be set to 'unknown'
  raw_bip = read_raw_bids(
/tmp/ipykernel_103803/1923545361.py:2: RuntimeWarning: DigMontage is only a subset of info. There are 141 channel positions not present in the DigMontage. The channels missing from the montage are:

['LPOG1-LPOG9', 'LPOG1-LPOG2', 'LPOG2-LPOG10', 'LPOG2-LPOG3', 'LPOG3-LPOG4', 'LPOG3-LPOG11', 'LPOG4-LPOG5', 'LPOG4-LPOG12', 'LPOG5-LPOG6', 'LPOG5-LPOG13', 'LPOG6-LPOG7', 'LPOG6-LPOG14', 'LPOG7-LPOG8', 'LPOG7-LPOG15', 'LPOG8-LPOG16', 'LPOG9-LPOG17', 'LPOG9-LPOG10', 'LPOG10-LPOG11', 'LPOG10-LPOG18', 'LPOG11-LPOG12', 'LPOG11-LPOG19', 'LPOG12-LPOG13', 'LPOG12-LPOG20', 'LPOG13-LPOG14', 'LPOG13-LPOG21', 'LPOG14-LPOG15', 'LPOG14-LPOG22', 'LPOG15-LPOG16', 'LPOG15-LPOG23', 'LPOG16-LPOG24

<RawEDF | sub-R1111M_ses-0_task-FR1_acq-bipolar_ieeg.edf, 141 x 1623500 (3247.0 s), ~114 kB, data not loaded>

In [14]:
import json

coordsys_path = base.copy().update(
    suffix="coordsystem",
    extension=".json",
    space="MNI152NLin6ASym"
).fpath

with open(coordsys_path, "r") as f:
    coordsys = json.load(f)

coordsys

{'iEEGCoordinateSystem': 'MNI152NLin6ASym', 'iEEGCoordinateUnits': 'mm'}

In [3]:
# let's find subjects who did the ValueCourier experiment
VC_df = df.query("experiment == 'ValueCourier'")
VC_df['subject'].unique()[:20]

array(['LTP001', 'LTP606', 'LTP607', 'LTP609', 'LTP610', 'LTP612',
       'LTP613', 'LTP614', 'LTP9992', 'LTP9993'], dtype=object)

In [4]:
# we'll pick LTP606 and select out this subject's ValueCourier sessions
sub_scalp = 'LTP606'
exp_scalp = 'ValueCourier'
df_select = df[(df['subject'] == sub_scalp) & (df['experiment'] == exp_scalp)]
display(df_select); print(f'{sub_scalp} sessions: {np.array(df_select.session)}')

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
7696,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,0,NaN,NaN,0,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7697,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,1,NaN,NaN,1,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7698,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,2,NaN,NaN,2,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7699,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,3,NaN,NaN,3,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7700,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,4,NaN,NaN,4,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...
7701,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...,NaN,ValueCourier,build,0,NaN,0,NaN,5,NaN,NaN,5,LTP606,LTP606,NaN,protocols/ltp/subjects/LTP606/experiments/Valu...


LTP606 sessions: [0 1 2 3 4 5]


In [5]:
# lets load the data from the first session
df_sess = df_select.iloc[0]         # select 1 row

# instantiate a Reader object using session metadata
# subjects beginning with 'LTP' are scalp subjects, so we don't need to specify the localization and montage
cmlreader = cml.CMLReader(subject=df_sess['subject'], experiment=df_sess['experiment'], session=df_sess['session'])
# load the behavioral events
evs_cml = cmlreader.load('events')
evs_cml[:10]

,eegoffset,actualvalue,compensation,correctPointingDirection,eegfile,eogArtifact,experiment,finalrecalled,intruded,intrusion,item,itemno,itemvalue,montage,msoffset,mstime,multiplier,numingroupchosen,phase,playerrotY,presX,presZ,primacybuf,protocol,recalled,recencybuf,rectime,serialpos,session,store,storeX,storeZ,storepointtype,subject,submittedPointingDirection,trial,type,valuerecall
0,23440,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761665861795,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,store mappings,-999
1,31922,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-1,-999,0,-1,1761665865937,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,VIDEO_START,-999
2,468068,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-1,-999,0,-1,1761666078916,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,VIDEO_STOP,-999
3,646089,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666165847,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
4,646183,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666165893,0.846154,4,1,-999.0,-4.949219,10.500000,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,TL_START,-999
5,1724585,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666692498,0.846154,4,1,-999.0,-3.000000,8.757812,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,TL_END,-999
6,1725717,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666693051,0.846154,4,1,-999.0,-3.937500,11.757812,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
7,1731210,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666695733,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
8,1763086,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666711299,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999
9,1766295,-999.0,8.461538,-999,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,-999,-999,-999,-999,-999,-999,0,-1,1761666712866,0.846154,4,1,-999.0,-3.953125,11.804688,2,ltp,-999,3,-999,-999,0,-999,-999.0,-999.0,,LTP606,-999,0,SESS_START,-999


### OpenBIDS

To load behavioral data in OpenBIDS, we first set the root of the bids database. The database is structured as such:

<pre>
root/
├── sub-{subject_id}/
│   └── ses-{session_id}/
│       ├── beh/
│       │   ├── sub-{subject_id}_ses-{session_id}_task-{experiment}_beh.json
│       │   └── sub-{subject_id}_ses-{session_id}_task-{experiment}_beh.tsv
│       └── eeg/
│           ├── sub-{subject_id}_ses-{session_id}_task-{experiment}_eeg.bdf
│           └── sub-{subject_id}_ses-{session_id}_task-{experiment}_events.tsv
└── participants.tsv
</pre>

We can use the get_entity_vals functon from the mne_bids package to find all the subjects in the database. Once we have selected a subject, we can use the same function to find all sessions and experiments (called 'task' in OpenBIDS format) associated with that subject. Finally, we can find the .tsv file containing the behavioral data and load it into a Pandas Dataframe.

In [17]:
# set root
bids_root = "/data/LTP_BIDS/"

In [18]:
# let's find subjects loaded in the database
subjects = get_entity_vals(bids_root, "subject")
subjects

['LTP063',
 'LTP064',
 'LTP065',
 'LTP066',
 'LTP067',
 'LTP068',
 'LTP069',
 'LTP070',
 'LTP073',
 'LTP074',
 'LTP093',
 'LTP106',
 'LTP115',
 'LTP117',
 'LTP122',
 'LTP123',
 'LTP133',
 'LTP138',
 'LTP187',
 'LTP207',
 'LTP229',
 'LTP246',
 'LTP260',
 'LTP312',
 'LTP327',
 'LTP329',
 'LTP339',
 'LTP347',
 'LTP354',
 'LTP355',
 'LTP606',
 'LTP607',
 'LTP609',
 'LTP610',
 'LTP612',
 'LTP613',
 'LTP614']

In [5]:
# now get the root of the subject and find which experiments they have completed
subject_root = os.path.join(bids_root, f"sub-LTP606")
tasks = get_entity_vals(subject_root, "task")
tasks

['valuecourier']

In [6]:
# we can also see how many sessions they have completed
sessions = get_entity_vals(subject_root, "session")
sessions

['0', '1', '2', '3', '4', '5']

In [60]:
# plug in the subject, task, and session info into BIDSPath and have the datatype set to "beh" to get the path to the .tsv file
# all inputs to BIDSPath should be strings so convert using the str() function
# load it using read_csv
subject = "LTP606"
task = tasks[0]
session = sessions[0]

path_bids = BIDSPath(
                subject=subject,
                session=str(session),
                task=task,  
                datatype="beh", 
                suffix="beh",
                extension=".tsv",
                root='/data/LTP_BIDS'
            )
evs_bids = pd.read_csv(path_bids.fpath, sep="\t")
evs_bids[:10]
evs_bids.columns

Index(['mstime', 'trial_type', 'stim_file', 'actualvalue', 'compensation',
       'eegfile', 'eogArtifact', 'experiment', 'intruded', 'intrusion', 'item',
       'itemno', 'itemvalue', 'montage', 'msoffset', 'multiplier',
       'numingroupchosen', 'phase', 'playerrotY', 'presX', 'presZ',
       'primacybuf', 'protocol', 'recalled', 'rectime', 'recencybuf',
       'serialpos', 'session', 'store', 'storeX', 'storeZ', 'storepointtype',
       'subject', 'trial', 'valuerecall'],
      dtype='object')

In [53]:
# we can also set the datatype to "eeg" and load it this way
bids_path = BIDSPath(
    subject=subject,
    session=str(session),
    task=task,
    datatype="eeg",
    root='/data/LTP_BIDS',
)

events_path = os.path.join(bids_path.directory, bids_path.basename + "_events.tsv")
evs_bids = pd.read_csv(events_path, sep="\t")
evs_bids[:10]

,onset,duration,trial_type,sample,stim_file,actualvalue,compensation,eegfile,eogArtifact,experiment,intruded,intrusion,item,itemno,itemvalue,montage,msoffset,multiplier,numingroupchosen,phase,playerrotY,presX,presZ,primacybuf,protocol,recalled,rectime,recencybuf,serialpos,session,store,storeX,storeZ,storepointtype,subject,trial,valuerecall
0,11.445312,NaN,store mappings,23440,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
1,15.586914,NaN,VIDEO_START,31922,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,-1.0,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
2,228.548828,NaN,VIDEO_STOP,468068,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,-1.0,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
3,315.473145,NaN,SESS_START,646089,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
4,315.519043,NaN,TL_START,646183,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-4.949219,10.500000,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
5,842.082520,NaN,TL_END,1724585,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.000000,8.757812,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
6,842.635254,NaN,SESS_START,1725717,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.937500,11.757812,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
7,845.317383,NaN,SESS_START,1731210,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
8,860.881836,NaN,SESS_START,1763086,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN
9,862.448730,NaN,SESS_START,1766295,NaN,NaN,8.461538,/protocols/ltp/subjects/LTP606/experiments/Val...,-1,ValueCourier,NaN,NaN,NaN,NaN,NaN,0,-1,0.846154,4.0,1,NaN,-3.953125,11.804688,2.0,ltp,NaN,NaN,3.0,NaN,0,NaN,NaN,NaN,NaN,LTP606,0,NaN


In [59]:
evs_bids.onset

0        11.445312
1        15.586914
2       228.548828
3       315.473145
4       315.519043
          ...     
237    5258.713379
238    5258.714355
239    5260.779297
240    5260.782227
241    5260.783203
Name: onset, Length: 242, dtype: float64

## Differences Between CML and OpenBIDS Events

The event dataframes loaded via OpenBIDS and CMLReader are identical in content, with the only differences being a small number of additional or renamed columns in the OpenBIDS version that conform to OpenBIDS formatting standards.

In [ ]:
evs_cml.columns

In [ ]:
evs_bids.columns

### Column differences
| CMLReader column | OpenBIDS column | Notes |
|------------------|-----------------|-------|
| `eegoffset` | `sample` | Same information; renamed to match BIDS conventions |
| `type` | `trial_type` | Renamed for BIDS compliance |
| `mstime` | `mstime` | In BIDS, `mstime` is typically shifted so that `mstime[0] = 0` |
| *(derived)* | `onset` | Computed as `sample / sample_frequency` |
| *(not present)* | `duration` | Not used / not applicable |
| *(not present)* | `stim_file` | Empty string placeholder required by BIDS |
                          

## Loading Raw EEG

Here's how we load the whole raw EEG file without creating epochs.

### CMLReader

In [ ]:
# use reader from events
eeg_cml = cmlreader.load_eeg().to_ptsa()

### OpenBIDS

In [ ]:
# use BIDSPath with datatype set to "eeg"
bids_path = BIDSPath(
            subject=subject,
            session=str(session),
            task=task,
            datatype="eeg",
            root=bids_root,
        )

# load the header from the bids file
raw = read_raw_bids(
    bids_path,
    verbose=True,
)

In [ ]:
# ptsa is just a wrapper for XArray, so we can just create an XArray object using the data from raw
eeg_bids = xr.DataArray(
    raw.get_data()[None, :, :],                # load the raw eeg data                  
    dims=("event", "channel", "time"),      
    coords={
        "event": [0],                          # singleton event index
        "channel": raw.ch_names,
        "time": raw.times  ,
        "samplerate": raw.info["sfreq"],          
    },
    name="eeg",
)
eeg_bids

## Loading Epoched EEG

Here's how to load Epoched EEG data. First let's set the constants.

In [ ]:
# constants
REL_START, REL_STOP = 200, 3000
BUFFER_MS = 1000
WIDTH = 6

FREQS = np.logspace(np.log10(2), np.log10(100), 46)
NOTCH_BAND = (58., 62.)
BATCH_EVENTS = 64

### CMLReader

It is easy to load the eeg data using the load_eeg method of the Reader object.

In [ ]:
# many of our analyses use the clean function, but to get the same data as the BIDS loader, we will forego it
# epochs_cml = reader.load_eeg(
#     word_evs_cml, rel_start=-BUFFER_MS, rel_stop=REL_STOP + BUFFER_MS, clean='LCF'
# ).to_ptsa()

# filter events
word_evs_cml = evs_cml[evs_cml["type"] == "WORD"]

epochs_cml = cmlreader.load_eeg(
    word_evs_cml, rel_start=-BUFFER_MS, rel_stop=REL_STOP + BUFFER_MS
).to_ptsa()
epochs_cml

### OpenBIDS

We can load epochs for OpenBIDS using the MNE Epochs object then converting to ptsa.

REMINDER: MNE Epochs uses seconds so we must divide by 1000 if the constants were in seconds.

In [ ]:
# get BIDSPath
bids_path = BIDSPath(
    subject=subject,
    session=str(session),
    task=task,
    datatype="eeg",
    root=bids_root,
)

# and load raw data
raw = read_raw_bids(bids_path)

# set non-EEG channels
raw.set_channel_types({
    "EXG1": "eog", "EXG2": "eog", "EXG3": "eog", "EXG4": "eog",
    "EXG5": "misc", "EXG6": "misc", "EXG7": "misc", "EXG8": "misc",
})

### Filtering Events for Loading

In order to select the events that we will epoch over, we must get the string representation from the events from the raw data's header using the events_from_annotations function. The function returns "events" which maps the sample to the event index and "event_id" which maps each event name to its index.

In [ ]:
# get events from raw data's header
events, event_id = mne.events_from_annotations(raw)

To filter the events, we select only the event_ids we are interested in.

In [ ]:
# filter word events and return the index of the WORD event
word_event_id = {k: v for k, v in event_id.items() if k == "WORD"}
word_event_id

In [ ]:
tmin = (-BUFFER_MS / 1000)
tmax = ((REL_STOP /1000 + BUFFER_MS / 1000))

# load into MNE Epochs object
epochs_mne = mne.Epochs(
    raw,
    events=events,                      # we can now load the events here
    event_id=word_event_id,             # we cna also load the filtered events here
    tmin=tmin,
    tmax=tmax,
    baseline=(None, 0), # None goes to start, 0 is onset of stimuli
    preload=True,
    event_repeated="merge",
)
epochs_mne

In [ ]:
# convert to ptsa
epochs_bids = TimeSeries.from_mne_epochs(epochs_mne, word_evs_bids)
epochs_bids